<a href="https://colab.research.google.com/github/XdstruCTor/nlp/blob/master/CFG_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
import nltk
from nltk import CFG
nltk.download('treebank')
from nltk.tree import Tree
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### CKY Parsing Algorithm

The CKY (Cocke-Kasami-Younger) algorithm is a bottom-up parsing algorithm used to determine whether a given sentence can be generated by a context-free grammar (CFG) in Chomsky Normal Form (CNF). Here, we implement the CKY algorithm for a sentence and CFG. Briefly the algorithm works by building up parse trees from individual words, combining them into larger subtrees until the entire sentence is parsed.

#### 1. **Grammar Definition**

CFG in Chomsky Normal Form (CNF):

In [ ]:


grammar = CFG.fromstring("""
  S -> NP VP
  NP -> A NP
  NP -> A N
  A -> 'colorless'|'green'
  N -> 'ideas'
  VP -> V Adv
  V -> 'sleep'
  Adv -> 'furiously'
""")

grammar.chomsky_normal_form()

<Grammar with 9 productions>

We then convert the grammar into CNF by calling grammar.chomsky_normal_form(). since the CKY algorithm works only with grammars in CNF, which means that all rules must either:

   - Produce exactly two non-terminal symbols on the right-hand side (A -> B C), or  
   - Produce a single terminal symbol (A -> 'terminal').

### **Flipping the Grammar for CKY Parsing**
In CKY, the parsing table needs access to the rules in reverse form (right-hand side as keys). This way, we can easily look up what non-terminal could have produced a given pair of non-terminals. To achieve this, the grammar is flipped:

In [ ]:
def get_flipped_grammar(grammar):
  rules = grammar.productions()
  flipped_grammar = {}
  for rule in rules:
    rhs = rule.rhs()
    if rhs not in flipped_grammar:
      flipped_grammar[rhs] = set()
    flipped_grammar[rhs].add(rule.lhs())
  return flipped_grammar

flipped_grammar = get_flipped_grammar(grammar)
for rhs, lhs_set in flipped_grammar.items():
    print(f"{rhs} -> {', '.join(str(lhs) for lhs in lhs_set)}")

(NP, VP) -> S
(A, NP) -> NP
(A, N) -> NP
('colorless',) -> A
('green',) -> A
('ideas',) -> N
(V, Adv) -> VP
('sleep',) -> V
('furiously',) -> Adv


## **Explanation of the CKY function**

We initialize a 2D table (`table`) with dimensions (`n+1`) `x` (`n+1`) (where `n` is the number of words in the sentence). Each entry in the table stores a list of possible parse trees that can be derived for a substring of the sentence.
   - Then the first step is to fill in the table for `terminals`. For each word in the sentence, we check if it's in the grammar's right-hand side and add the corresponding non-terminal.  
     - Next, for each span of words, we try to combine previously computed entries (subtrees) from smaller spans. For each combination of subtrees, we check the grammar to see if they can be combined into a larger subtree

Once the table is filled, the possible parse trees for the whole sentence (from the first word to the last) will be stored in table `[0]` `[n]`. then print these trees
        

In [ ]:
def cky(words, flipped_grammar):
    len_1 = len(words) + 1
    #table initialisation
    table = [[[] for _ in range(len_1)] for _ in range(len_1)]  # Corrected table initialization

    # Fill in the terminal rules
    for col in range(1, len_1):
        word = words[col - 1]
        col_tuple = (word,)  # Using tuple directly
        if col_tuple in flipped_grammar:
            for symbol in flipped_grammar[col_tuple]:
                table[col - 1][col].append(Tree(symbol, [word]))
    # Filling in the grammar rules
    for col in range(1, len_1):
        for row in range(col - 1, -1, -1):
            for pivot in range(row + 1, col):
                for subset1 in table[row][pivot]:
                    for subset2 in table[pivot][col]:
                        rhs = (subset1.label(), subset2.label())
                        if rhs in flipped_grammar:
                            for left_symbol in flipped_grammar[rhs]:
                                table[row][col].append(Tree(left_symbol, [subset1, subset2]))
            print(f"Filled table[{row}][{col}]: {table[row][col]}")

        trees = table[0][-1]
    for tree in trees:
        print(tree.pformat())
    return trees

words = word_tokenize('colorless green ideas sleep furiously')
parsing_trees = cky(words, flipped_grammar)

if parsing_trees:
    print(f"The sentence '{' '.join(words)}' can be generated by the grammar")
else:
    print(f"The sentence '{' '.join(words)}' cannot be generated by the grammar")

tree = Tree.fromstring(str(parsing_trees[0]))
tree.pretty_print()

Filled table[0][1]: [Tree(A, ['colorless'])]
Filled table[1][2]: [Tree(A, ['green'])]
Filled table[0][2]: []
Filled table[2][3]: [Tree(N, ['ideas'])]
Filled table[1][3]: [Tree(NP, [Tree(A, ['green']), Tree(N, ['ideas'])])]
Filled table[0][3]: [Tree(NP, [Tree(A, ['colorless']), Tree(NP, [Tree(A, ['green']), Tree(N, ['ideas'])])])]
Filled table[3][4]: [Tree(V, ['sleep'])]
Filled table[2][4]: []
Filled table[1][4]: []
Filled table[0][4]: []
Filled table[4][5]: [Tree(Adv, ['furiously'])]
Filled table[3][5]: [Tree(VP, [Tree(V, ['sleep']), Tree(Adv, ['furiously'])])]
Filled table[2][5]: []
Filled table[1][5]: [Tree(S, [Tree(NP, [Tree(A, ['green']), Tree(N, ['ideas'])]), Tree(VP, [Tree(V, ['sleep']), Tree(Adv, ['furiously'])])])]
Filled table[0][5]: [Tree(S, [Tree(NP, [Tree(A, ['colorless']), Tree(NP, [Tree(A, ['green']), Tree(N, ['ideas'])])]), Tree(VP, [Tree(V, ['sleep']), Tree(Adv, ['furiously'])])])]
(S
  (NP (A colorless) (NP (A green) (N ideas)))
  (VP (V sleep) (Adv furiously)))
The se

CKY is efficient for grammars in CNF since it operates in `O(n^3)` time, with `n` the number of words in the sentence.

# **Earley algorithm**

### Explanation of the Earley Algorithm

The **Earley parser** is an algorithm designed for parsing sentences using context-free grammars, which can handle a wide range of language structures, including ambiguous and recursive grammars. It builds a chart (a dynamic programming table) where each entry tracks the parsing progress at a certain point in the input sentence.

#### **Working of the Algorithm:**

1. **State**: Each state in the chart represents a step in the parsing process. It consists of:
   - **Label**: A non-terminal.
   - **Rules**: The current production rule being processed.
   - **Dot (•)**: Marks the current position in the production rule. The dot indicates what has been processed and what remains.
   - **Start and End indices**: The range in the input sentence this rule applies to.
   - **Made from**: The state this state was derived from (used for backtracking in parsing).
   - **Producer**: How the state was created (predictor, scanner, completer).

2. **Operations**:
   - **Predictor**: Adds possible expansions for non-terminals.
   - **Scanner**: Matches terminals (words from the input sentence).
   - **Completer**: Combines parsed structures when non-terminals are completed.

## Base case implementation of the Earley algorithm


### Sample grammar

In [ ]:
sample_grammar = {
    '<start>': [['<A>','<B>']],
    '<A>': [['a', '<B>', 'c'], ['a', '<A>']],
    '<B>': [['b', '<C>'], ['<D>']],
    '<C>': [['c']],
    '<D>': [['d']]
}


### Column Data structure

In [ ]:
class Column:
  def __init__(self, index, letter):
    self.index, self.letter = index, letter
    self.states, self._unique = [], {}
  def __str__(self):
    return "% chart[%d]\n%s" % (self.letter, self.index, "\n".join(
        str(state) for state in self.states if state.finished()))
  def to_repr(self):
    return "%s chart[%d]\n%s" % (self.letter, self.index, "\n".join(
        str(state) for state in self.states))
  def add(self, state):
    if state in self._unique:
      return self._unique[state]
    self._unique[state] = state
    self.states.append(state)
    return self._unique[state]


### The State Data Structure

In [ ]:
class State:
  def __init__(self, name, expr, dot, s_col, e_col=None):
    self.name, self.expr, self.dot = name, expr, dot
    self.s_col, self.e_col = s_col, e_col

  def finished(self):
    return self.dot >= len(self.expr)

  def at_dot(self):
    return self.expr[self.dot] if self.dot < len(self.expr) else None

  def __str__(self):
    def idx(var):
      return var.index if var else -1
    return show_dot(self.name, self.expr, self.dot)

  def copy(self):
    return State(self.name, self.expr, self.dot, self.s_col.index)

  def _t(self):
    return(self.name, self.expr, self.dot, self.s_col.index)

  def __hash__(self):
    return hash(self._t())

  def __eq__(self, other):
    return self.t()== other._t()

  def advance(self):
    return State(self.name, self.expr, self.dot + 1, self.s_col)

def show_dot(sym, rule, pos, dotstr= "|", extents=''):
  extends = str(extents)
  return sym + '::=' + ' '.join([
      str(p)
      for p in [*rule[0:pos], dotstr, *rule[pos:]]]) + extents




### The parser is able to move from B to D (bellow) which is the next logical step according to the sample grammar

In [ ]:
nt_name = '<B>'
nt_expr = tuple(sample_grammar[nt_name][1])
col_0 = Column(0, None)
a_state = State(nt_name, tuple(nt_expr), 0, col_0)
print(a_state.at_dot())

<D>


## **Implementation of Earley Parsing algorithm general case**

In [ ]:
class State:
  #  initializing a new State object with its properties
  # This class represents a single state in the Earley chart
  def __init__(self, label, rules, dot_idx, start_idx, end_idx, idx, made_from, producer):
    self.label = label # The non-terminal symbol on the left hand side of the rule
    self.rules = rules #
    self.dot_idx = dot_idx # Position of the dot in the dot in the rule (progress)
    self.start_idx = start_idx # Start position of the rule in the sentence
    self.end_idx = end_idx # End position of the rule
    self.idx = idx # Unique id for the state
    self.made_from = made_from # List of states derived from previous state
    self.producer = producer # Operation that create the state

  def next(self):
    """Returns the tag of the dot"""
    if self.dot_idx < len(self.rules):
      return self.rules[self.dot_idx]
    return None

  # Checks if the state is complete (dot(bullet) is at the end of the rule)
  def complete(self):
    return len(self.rules) == self.dot_idx
  # Compare two State objects for equality
  def __eq__(self, other):
    return (self.label == other.label and
            self.rules == other.rules and
            self.dot_idx == other.dot_idx and
            self.start_idx == other.start_idx and
            self.end_idx == other.end_idx)

  # represent a Stste object as a string
  def __str__(self):
    rule_string = ''
    for i, rule in enumerate(self.rules):
      if i == self.dot_idx:
          rule_string += '• '
      rule_string += rule + ' '
    if self.dot_idx == len(self.rules):
      rule_string += '• '
    return 'S%d %s -> %s [%d, %d] %s %s' % (self.idx, self.label, rule_string, self.start_idx,
                                            self.end_idx, self.made_from, self.producer)

# Earley logic implementation
class Earley:
  def __init__(self, words, grammar, terminals):
    self.chart = [[] for _ in range(len(words) + 1)]
    self.current_id = 0
    self.words = words # input sentence as a list of words
    self.grammar = grammar # Arbritrary grammar rules
    self.terminals = terminals # list of terminals in the grammar

  def get_new_id(self):
    self.current_id += 1
    return self.current_id -1

  def is_terminal(self, tag):
    return tag in self.terminals

  def is_complete(self, state):
    return len(state.rules) ==  state.dot_idx

  def enqueue(self, state, chart_entry):
    if state not in self.chart[chart_entry]:
      self.chart[chart_entry].append(state)
    else:
      self.current_id -= 1

  def predictor(self, state):
    next_item = state.next()
    if next_item in self.grammar:
      for production in self.grammar[next_item]:
        self.enqueue(State(state.next(), production, 0, state.end_idx, state.end_idx, self.get_new_id(), [], 'predictor'), state.end_idx)

  def scanner(self, state):
    if self.words[state.end_idx] in self.grammar[state.next()]:
        self.enqueue(State(state.next(), [self.words[state.end_idx]], 1, state.end_idx, state.end_idx + 1, self.get_new_id(), [], 'scanner'), state.end_idx + 1)

  def completer(self, state):
    for s in self.chart[state.start_idx]:
      if not s.complete() and s.next() == state.label and s.end_idx == state.start_idx and s.label != 'gamma':
        self.enqueue(State(s.label, s.rules, s.dot_idx + 1, s.start_idx, state.end_idx, self.get_new_id(), s.made_from + [state.idx], 'completer'), state.end_idx)

  def parse(self):
    self.enqueue(State('gamma', ['S'], 0, 0, 0, self.get_new_id(), [], 'dummy start state'),0)

    for i in range(len(self.words)+ 1):
      for state in self.chart[i]:
        if not state.complete() and not self.is_terminal(state.next()):
          self.predictor(state)
        elif i != len(self.words) and not state.complete() and self.is_terminal(state.next()):
          self.scanner(state)
        else:
          self.completer(state)

  def __str__(self):
      res = ' '

      for i, chart in enumerate(self.chart):
        res += '\nChart[%d]\n' % i
        for state in chart:
          res += str(state) + '\n'

      return res


In [ ]:
def text():

  grammar = {
        'S':           [['NP', 'VP'], ['Aux', 'NP', 'VP'], ['VP']],
        'NP':          [['Det', 'Nominal'], ['Proper-Noun']],
        'Nominal':     [['Noun'], ['Noun', 'Nominal']],
        'VP':          [['Verb'], ['Verb', 'NP']],
        'Det':         ['the', 'a'],
        'Noun':        ['dog', 'cat', 'book', 'flight'],
        'Verb':        ['sees', 'chases', 'book'],
        'Aux':         ['does'],
        'Proper-Noun': ['Houston', 'TWA']
  }

  terminals = ['Det', 'Noun', 'Verb', 'Aux', 'Proper-Noun']

  words = ['the', 'dog', 'sees', 'the', 'cat']

  earley = Earley(words, grammar, terminals)
  earley.parse()
  print(earley)

if __name__ == '__main__':
  text()

 
Chart[0]
S0 gamma -> • S  [0, 0] [] dummy start state
S1 S -> • NP VP  [0, 0] [] predictor
S2 S -> • Aux NP VP  [0, 0] [] predictor
S3 S -> • VP  [0, 0] [] predictor
S4 NP -> • Det Nominal  [0, 0] [] predictor
S5 NP -> • Proper-Noun  [0, 0] [] predictor
S6 VP -> • Verb  [0, 0] [] predictor
S7 VP -> • Verb NP  [0, 0] [] predictor

Chart[1]
S8 Det -> the •  [0, 1] [] scanner
S9 NP -> Det • Nominal  [0, 1] [8] completer
S10 Nominal -> • Noun  [1, 1] [] predictor
S11 Nominal -> • Noun Nominal  [1, 1] [] predictor

Chart[2]
S12 Noun -> dog •  [1, 2] [] scanner
S13 Nominal -> Noun •  [1, 2] [12] completer
S14 Nominal -> Noun • Nominal  [1, 2] [12] completer
S15 NP -> Det Nominal •  [0, 2] [8, 13] completer
S16 Nominal -> • Noun  [2, 2] [] predictor
S17 Nominal -> • Noun Nominal  [2, 2] [] predictor
S18 S -> NP • VP  [0, 2] [15] completer
S19 VP -> • Verb  [2, 2] [] predictor
S20 VP -> • Verb NP  [2, 2] [] predictor

Chart[3]
S21 Verb -> sees •  [2, 3] [] scanner
S22 VP -> Verb •  [2, 3] [2

#### **Chart Breakdown:**

Each chart represents the state of parsing at a particular position in the input sentence. The goal is to track the parsing progress using a sequence of states in these charts.

### **Chart[0]**


Chart[0]
S0 gamma -> • S  [0, 0] [ ] dummy start state  
S1 S -> • NP VP  [0, 0] [ ] predictor  
S2 S -> • Aux NP VP  [0, 0] [] predictor  
S3 S -> • VP  [0, 0] [ ] predictor  
S4 NP -> • Det Nominal  [0, 0] [ ] predictor  
S5 NP -> • Proper-Noun  [0, 0] [ ] predictor  
S6 VP -> • Verb  [0, 0] [ ] predictor  
S7 VP -> • Verb NP  [0, 0] [ ] predictor


- **Chart[0]** shows the initial prediction phase (`predictor`), where the parser expands possible rules that could start the sentence. `S -> NP VP`, `S -> VP` and `Aux NP VP` represent possible sentence structures.
- The dummy start state `S0 gamma -> bullet S` is used to initialize the parsing process.

### **Chart[1]**
S8 Det -> the •  [0, 1] [ ] scanner  
S9 NP -> Det • Nominal  [0, 1] [8] completer  
S10 Nominal -> • Noun  [1, 1] [ ] predictor  
S11 Nominal -> • Noun Nominal  [1, 1] [ ] predictor


- **Chart[1]** shows the effect of reading the first word `the` (`scanner`). After scanning `the`, the state `Det -> the •` moves the dot past `the`, indicating this terminal has been successfully matched.
- Next, the parser predicts that the `NP` (noun phrase) continues with `Nominal`, so the dot moves forward in `NP -> Det Nominal`.

### **Chart[2]**
S12 Noun -> dog •  [1, 2] [ ] scanner  
S13 Nominal -> Noun •  [1, 2] [12] completer  
S14 Nominal -> Noun • Nominal  [1, 2] [12] completer  
S15 NP -> Det Nominal •  [0, 2] [8, 13] completer  
S16 Nominal -> • Noun  [2, 2] [ ] predictor  
S17 Nominal -> • Noun Nominal  [2, 2] [ ] predictor  
S18 S -> NP • VP  [0, 2] [15] completer  
S19 VP -> • Verb  [2, 2] [ ] predictor  
S20 VP -> • Verb NP  [2, 2] [ ] predictor


- **Chart[2]** continues after matching `dog`. This is processed by the scanner (`Noun -> dog •`), and now we can mark the `NP -> Det Nominal` as complete, since both `Det` and `Nominal` have been parsed.
- Now the parser predicts the next part of the sentence, such as expecting a verb phrase (`VP`).

### **Chart[3]**


S21 Verb -> sees •  [2, 3] [ ] scanner  
S22 VP -> Verb •  [2, 3] [21] completer  
S23 VP -> Verb • NP  [2, 3] [21] completer  
S24 S -> NP VP •  [0, 3] [15, 22] completer  
S25 NP -> • Det Nominal  [3, 3] [ ] predictor  
S26 NP -> • Proper-Noun  [3, 3] [ ] predictor


- **Chart[3]** completes the verb `sees` (`Verb -> sees •`) and moves forward to complete `VP -> Verb`.
- The sentence `S -> NP VP` is now complete, indicating that up to this point, the phrase `the dog sees` is valid.
- The parser also predicts possible noun phrases (`NP -> Det Nominal`) starting at position 3.

### **Chart[4]**

S27 Det -> the •  [3, 4] [ ] scanner  
S28 NP -> Det • Nominal  [3, 4] [27] completer  
S29 Nominal -> • Noun  [4, 4] [ ] predictor  
S30 Nominal -> • Noun Nominal  [4, 4] [ ] predictor  


- **Chart[4]** handles the next noun phrase, starting with `the` (`Det -> the •`). The parser continues predicting the next part of the noun phrase (`NP -> Det Nominal`).

### **Chart[5]**

S31 Noun -> cat •  [4, 5] [ ] scanner  
S32 Nominal -> Noun •  [4, 5] [31] completer  
S33 Nominal -> Noun • Nominal  [4, 5] [31] completer  
S34 NP -> Det Nominal •  [3, 5] [27, 32] completer  
S35 Nominal -> • Noun  [5, 5] [ ] predictor  
S36 Nominal -> • Noun Nominal  [5, 5] [ ] predictor  
S37 VP -> Verb NP •  [2, 5] [21, 34] completer  
S38 Nominal -> Noun Nominal •  [4, 5] [31, 35] completer  
S39 S -> NP VP •  [0, 5] [15, 37] completer

- **Chart[5]** finishes parsing the phrase `the cat` (`Noun -> cat •`), completing the noun phrase `Det Nominal`.
- The entire sentence `the dog sees the cat` is now parsed and completed (`S -> NP VP •`), marking the end of the successful parse.


- **Chart entries** represent the parser's progress, with the dot (`•`) indicating what part of the rule has been parsed.
- The **predictor** expands non-terminals to explore possible future rules.
- The **scanner** matches terminals (words in the input).
- The **completer** combines parsed subcomponents once a rule is fully matched.
- The goal is to complete a rule from the start symbol (`S`) covering the entire sentence (`gamma -> S •`), indicating that the sentence is valid according to the grammar.

### Earley Algorithm vs. CKY Algorithm

Both the **Earley** and **CKY** algorithms are parsing techniques used to determine whether a given sentence can be generated by a context-free grammar (CFG). However, they differ in terms of their approach, efficiency, and the types of grammars they can handle.

#### 1. **Differences**

- **Earley Algorithm**:
  - **Generalization**: Earley is a **general parsing algorithm** that can handle any context-free grammar, including ambiguous and non-CNF grammars. This makes it highly versatile.
  - **Efficiency**: It operates in **O(n^3)** time in the worst case, but for many grammars, especially those that are unambiguous, it can perform better in practice.
  - **Structure**: It uses a predictive parsing strategy, maintaining a chart of possible parses that evolves as the input is processed.

- **CKY Algorithm**:
  - **Specificity**: CKY is restricted to parsing **grammars in Chomsky Normal Form (CNF)**. This limits its applicability but also makes it simpler and more efficient for certain types of grammars.
  - **Efficiency**: The algorithm operates in **O(n^3)** time and is often faster than Earley for CNF grammars, especially when the grammar is not too complex.
  - **Table-based**: It builds a parsing table using dynamic programming, making it suitable for well-defined grammars.

#### 2. **Use Cases**

- **Earley Algorithm**:
  - Suitable for grammars that are ambiguous or not in CNF.
  - Ideal for natural language processing tasks where grammars may not be strictly defined.

- **CKY Algorithm**:
  - Best suited for formal grammars that can easily be converted into CNF.
  - Efficient for parsing in environments where grammar structure is strictly controlled.

In [ ]:
complex_sentence = "The quick brown fox jumps over the lazy dog while the cat watches carefully"

def text():
    grammar = {
        'S':           [['NP', 'VP'], ['S', 'Conj', 'S']],
        'NP':          [['Det', 'Nominal'], ['A', 'Nominal']],
        'Nominal':     [['Noun'], ['A', 'Nominal']],
        'VP':          [['Verb', 'Adv'], ['Verb', 'NP'], ['Verb', 'PP'], ['Verb', 'NP', 'Adv']],
        'PP':          [['P', 'NP']],
        'A':           ['quick', 'brown', 'lazy'],
        'Det':         ['the', 'a'],
        'Noun':        ['dog', 'cat', 'fox'],
        'Verb':        ['jumps', 'watches'],
        'P':           ['over'],
        'Conj':        ['while'],
        'Adv':         ['carefully'],
    }

    terminals = ['Det', 'Noun', 'Verb', 'Aux', 'Proper-Noun','A', 'P', 'carefully', 'Conj', 'Adv']

    words = ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', 'while', 'the', 'cat', 'watches', 'carefully']

    earley = Earley(words, grammar, terminals)
    parsing_result = earley.parse()
    print(earley)
if __name__ == '__main__':
    text()




 
Chart[0]
S0 gamma -> • S  [0, 0] [] dummy start state
S1 S -> • NP VP  [0, 0] [] predictor
S2 S -> • S Conj S  [0, 0] [] predictor
S3 NP -> • Det Nominal  [0, 0] [] predictor
S4 NP -> • A Nominal  [0, 0] [] predictor

Chart[1]
S5 Det -> the •  [0, 1] [] scanner
S6 NP -> Det • Nominal  [0, 1] [5] completer
S7 Nominal -> • Noun  [1, 1] [] predictor
S8 Nominal -> • A Nominal  [1, 1] [] predictor

Chart[2]
S9 A -> quick •  [1, 2] [] scanner
S10 Nominal -> A • Nominal  [1, 2] [9] completer
S11 Nominal -> • Noun  [2, 2] [] predictor
S12 Nominal -> • A Nominal  [2, 2] [] predictor

Chart[3]
S13 A -> brown •  [2, 3] [] scanner
S14 Nominal -> A • Nominal  [2, 3] [13] completer
S15 Nominal -> • Noun  [3, 3] [] predictor
S16 Nominal -> • A Nominal  [3, 3] [] predictor

Chart[4]
S17 Noun -> fox •  [3, 4] [] scanner
S18 Nominal -> Noun •  [3, 4] [17] completer
S19 Nominal -> A Nominal •  [2, 4] [13, 18] completer
S20 Nominal -> A Nominal •  [1, 4] [9, 19] completer
S21 NP -> Det Nominal •  [0, 4]

In [ ]:
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
sample_text = "the quick brown fox jumps over the lazy dog while the cat watches carefully"
token = word_tokenize(sample_text)
tagged = pos_tag(token)
tagged

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('the', 'DT'),
 ('quick', 'JJ'),
 ('brown', 'NN'),
 ('fox', 'NN'),
 ('jumps', 'VBZ'),
 ('over', 'IN'),
 ('the', 'DT'),
 ('lazy', 'JJ'),
 ('dog', 'NN'),
 ('while', 'IN'),
 ('the', 'DT'),
 ('cat', 'NN'),
 ('watches', 'NNS'),
 ('carefully', 'RB')]

In [ ]:
grammar = CFG.fromstring("""
  S -> NP VP
  S -> S Sub S
  NP -> D N2
  N2 -> Adj N2
  N2 -> N
  VP -> V NP
  VP -> V PP
  VP -> V AdvP
  PP -> P NP
  AdvP -> Adv
  Sub -> 'while'
  D -> 'the'
  Adj -> 'quick'
  Adj -> 'brown'
  Adj -> 'lazy'
  N -> 'fox'
  N -> 'dog'
  N -> 'cat'
  V -> 'jumps'
  V -> 'watches'
  P -> 'over'
  Adv -> 'carefully'
""")

flipped_grammar = get_flipped_grammar(grammar)


words = word_tokenize('the quick brown fox jumps over the lazy dog while the cat watches carefully')
parsing_trees = cky(words, flipped_grammar)

if parsing_trees:
    print(f"The sentence '{' '.join(words)}' can be generated by the grammar")
else:
    print(f"The sentence '{' '.join(words)}' cannot be generated by the grammar")



In [ ]:
import nltk
from nltk import CFG
# Create a context-free grammar
grammar = CFG.fromstring("""
  S -> NP VP
  S -> S Sub S
  NP -> D N2
  N2 -> Adj N2
  N2 -> N
  VP -> V NP
  VP -> V PP
  VP -> V AdvP
  PP -> P NP
  AdvP -> Adv
  Sub -> 'while'
  D -> 'the'
  Adj -> 'quick'
  Adj -> 'brown'
  Adj -> 'lazy'
  N -> 'fox'
  N -> 'dog'
  N -> 'cat'
  V -> 'jumps'
  V -> 'watches'
  P -> 'over'
  Adv -> 'carefully'
""")
parser = nltk.ChartParser(grammar)
sentence = "the quick brown fox jumps over the lazy dog while the cat watches carefully".split()

# Parse the sentence
trees = list(parser.parse(sentence))

if trees:
    print("Parsing successful!")
    for tree in trees:
        print(tree)
else:
    print("Parsing failed.")

Parsing successful!
(S
  (S
    (NP (D the) (N2 (Adj quick) (N2 (Adj brown) (N2 (N fox)))))
    (VP
      (V jumps)
      (PP (P over) (NP (D the) (N2 (Adj lazy) (N2 (N dog)))))))
  (Sub while)
  (S
    (NP (D the) (N2 (N cat)))
    (VP (V watches) (AdvP (Adv carefully)))))
